## Clara Cabañas Pujadas and Emilie Krutnes Engen

# THIRD ASSIGNMENT PART II

## Programming PCA in Spark

Starting the Spark context

In [3]:
import sys
import os
import os.path

SPARK_HOME = """C:/spark-2.1.0-bin-hadoop2.7""" #CHANGE THIS PATH TO YOURS!

sys.path.append(os.path.join(SPARK_HOME, "python", "lib", "py4j-0.10.4-src.zip")) #BEWARE WITH py4j version!!
sys.path.append(os.path.join(SPARK_HOME, "python", "lib", "pyspark.zip"))
os.environ["SPARK_HOME"] = SPARK_HOME

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("wind") \
    .config("spark.sql.caseSensitive", "false") \
    .getOrCreate()
    
sc = spark.sparkContext

Import libraries

In [4]:
# Import some libraries
import matplotlib.pyplot as plt 
# For plotting data
import numpy as np              
import os

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import PCA
from pyspark.ml import Pipeline

os.getcwd()

'C:\\Users\\Clara\\Documents'

Get the wind data

In [5]:
wind_sd = spark.read.csv(path="wind.csv", header=True, inferSchema=True)

In [2]:
#wind_sd.printSchema()

In [1]:
#wind_sd.take(3)

Create train, validation and test partitions of the data

In [21]:
# Creating the train, validation and test partitions
wind_train = wind_sd.filter(wind_sd['year'] <= 2006)
wind_val = wind_sd.filter(wind_sd['year'] > 2006).filter(wind_sd['year'] <= 2008)
wind_test = wind_sd.filter(wind_sd['year'] > 2008)

Remove the columns not needed

In [23]:
# Removing the 5 first columns - not needed
wind_train = wind_train.drop('steps', 'year', 'month', 'day', 'hour')
wind_val = wind_val.drop('steps', 'year', 'month', 'day', 'hour')
wind_test = wind_test.drop('steps', 'year', 'month', 'day', 'hour')

Use VectorAssembler to put together the input attributes

In [24]:
from pyspark.ml.feature import VectorAssembler

ignore = ['energy']
assembler = VectorAssembler(
    inputCols=[x for x in wind_train.columns if x not in ignore],
    outputCol='features')

wind_train = assembler.transform(wind_train).select(['energy', 'features'])

ignore = ['energy']
assembler = VectorAssembler(
    inputCols=[x for x in wind_val.columns if x not in ignore],
    outputCol='features')

wind_val = assembler.transform(wind_val).select(['energy', 'features'])

ignore = ['energy']
assembler = VectorAssembler(
    inputCols=[x for x in wind_test.columns if x not in ignore],
    outputCol='features')

wind_test = assembler.transform(wind_test).select(['energy', 'features'])

Import libraries needed

In [26]:
from pyspark.ml.feature import PCA
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator

Perform PCA

In [33]:
# Pipeline 1 = pca 5 components => selector 3 features => decision tree 

results = []

# Perform PCA selecting number of features between 5 and 100
for i in range(5, 100, 5):

    pca = PCA(k=i, inputCol="features")
    dt = DecisionTreeRegressor(featuresCol=pca.getOutputCol(), labelCol="energy")

    pipeline = Pipeline(stages=[pca, dt])

    model = pipeline.fit(wind_train)

    predictions = model.transform(wind_val)

    evaluator = RegressionEvaluator(
                labelCol="energy", 
                predictionCol="prediction", 
                metricName="mae")

    mae = evaluator.evaluate(predictions)
    
    results.append([i, mae])

results


min(results[1])

[[5, 469.96397500686055],
 [10, 470.60650390856256],
 [15, 474.7406058882686],
 [20, 474.84649722479975],
 [25, 474.4392560320692],
 [30, 474.4392560320692],
 [35, 447.9666423914338],
 [40, 447.84534776696665],
 [45, 446.74254591006724],
 [50, 448.43167475012575],
 [55, 448.43167475012575],
 [60, 449.695364817011],
 [65, 445.1972840388339],
 [70, 447.16127319306423],
 [75, 445.73909491083185],
 [80, 451.7444198943323],
 [85, 443.88448426773783],
 [90, 448.4568654434856],
 [95, 457.25423865382623]]

From the results we have that k=85 gives the lowest MAE.

In [44]:
k_min = 85

wind_train_and_val = wind_sd.filter(wind_sd['year'] <= 2008)
wind_train_and_val = wind_train_and_val.drop('steps', 'year', 'month', 'day', 'hour')

ignore = ['energy']
assembler = VectorAssembler(
    inputCols=[x for x in wind_train_and_val.columns if x not in ignore],
    outputCol='features')

wind_train_and_val = assembler.transform(wind_train_and_val).select(['energy', 'features'])
wind_train_and_val = wind_train_and_val.withColumnRenamed('energy', 'label')

dt = DecisionTreeRegressor()
model=dt.fit(wind_train_and_val)

predictions_sd = model.transform(wind_test)

evaluator = RegressionEvaluator(
labelCol="energy", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions_sd)

print(str(mae) +  " MAE with all the attributes.")

321.794387566 MAE with all the attributes.


In [ ]:
spark.stop()